## 1) Import required libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 2) Load dataset

In [3]:
recipes_df = pd.read_csv('../data/csv/RAW_recipes.csv')
recipes_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


## 3) Visualize Dataframe Information

In [4]:
print(f'🔹 Dataset shape: {recipes_df.shape}')
print(f'🔹 Dataset dimensions: {recipes_df.columns}')
recipes_df.info()

🔹 Dataset shape: (231637, 12)
🔹 Dataset dimensions: Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), o

## 4) Feature Selection
Retain relevant features only

***Nutrition Information***
Nutrition information (calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat

In [5]:
recipes_df = recipes_df[['name', 'tags', 'nutrition', 'ingredients']]
recipes_df.head()

,name,tags,nutrition,ingredients
0,arriba baked winter squash mexican style,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]","['winter squash', 'mexican seasoning', 'mixed ..."
1,a bit different breakfast pizza,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]","['prepared pizza crust', 'sausage patty', 'egg..."
2,all in the kitchen chili,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]","['ground beef', 'yellow onions', 'diced tomato..."
3,alouette potatoes,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]","['spreadable cheese with garlic and herbs', 'n..."
4,amish tomato ketchup for canning,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]","['tomato juice', 'apple cider vinegar', 'sugar..."


#### Divide nutrition and ingredients into separate columns

In [6]:
import ast  # Safely evaluates strings like list literals

# Step 1: Convert string representation to actual list
recipes_df['nutrition'] = recipes_df['nutrition'].apply(ast.literal_eval)

# Step 2: Expand only kcal, fat, protein (skip others)
nutrition_expanded = pd.DataFrame(recipes_df['nutrition'].tolist(), columns=[
    'calories',     # index 0
    'fat_g',        # index 1
    None,           # sugar_pdv (ignored)
    None,           # sodium_pdv (ignored)
    'protein_g',    # index 4
    None,           # sat_fat_pdv (ignored)
    None            # carbs (unreliable, so we calculate it)
])

# Step 3: Drop rows with missing core values
nutrition_expanded = nutrition_expanded.dropna(subset=['calories', 'fat_g', 'protein_g'])

# Step 4: Derive carbs (estimate)
nutrition_expanded['carbohydrates_g'] = (
    nutrition_expanded['calories'] - (
        nutrition_expanded['protein_g'] * 4 + nutrition_expanded['fat_g'] * 9
    )
) / 4

# Step 5: Replace any negative values with 0
nutrition_expanded['carbohydrates_g'] = nutrition_expanded['carbohydrates_g'].clip(lower=0)

# Optional: Round for readability
nutrition_expanded = nutrition_expanded.round(2)

# Preview
nutrition_expanded.head()

# Reset index to ensure safe concat
recipes_df = recipes_df.reset_index(drop=True)
nutrition_expanded = nutrition_expanded.reset_index(drop=True)

# Merge horizontally
recipes_df_cleaned = pd.concat([recipes_df.drop(columns=['nutrition']), nutrition_expanded], axis=1)

# Drop unnecessary dimensions
recipes_df_cleaned = recipes_df_cleaned[['name', 'tags', 'ingredients', 'calories', 'fat_g', 'protein_g', 'carbohydrates_g']]
# Preview result
recipes_df_cleaned.head()

,name,tags,ingredients,calories,fat_g,protein_g,carbohydrates_g
0,arriba baked winter squash mexican style,"['60-minutes-or-less', 'time-to-make', 'course...","['winter squash', 'mexican seasoning', 'mixed ...",51.5,0.0,2.0,10.88
1,a bit different breakfast pizza,"['30-minutes-or-less', 'time-to-make', 'course...","['prepared pizza crust', 'sausage patty', 'egg...",173.4,18.0,22.0,0.00
2,all in the kitchen chili,"['time-to-make', 'course', 'preparation', 'mai...","['ground beef', 'yellow onions', 'diced tomato...",269.8,22.0,39.0,0.00
3,alouette potatoes,"['60-minutes-or-less', 'time-to-make', 'course...","['spreadable cheese with garlic and herbs', 'n...",368.1,17.0,14.0,39.78
4,amish tomato ketchup for canning,"['weeknight', 'time-to-make', 'course', 'main-...","['tomato juice', 'apple cider vinegar', 'sugar...",352.9,1.0,3.0,82.98


Encode ingredients as binary columns to ensure consistency across dataframes

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
import ast

# Step 1: Ensure ingredients column is list, not string
recipes_df_cleaned['ingredients'] = recipes_df_cleaned['ingredients'].apply(ast.literal_eval)

# Step 2: Clean and flatten all ingredients
def clean_ingredients(ingredient_list):
    return [
        ing.strip().strip("'\"").lower()
        for ing in ingredient_list
        if isinstance(ing, str) and ing.strip()
    ]

recipes_df_cleaned['ingredients_cleaned'] = recipes_df_cleaned['ingredients'].apply(clean_ingredients)

# Flatten and count ingredients
all_ingredients = [ing for sublist in recipes_df_cleaned['ingredients_cleaned'] for ing in sublist]
top_ingredients = [item for item, count in Counter(all_ingredients).most_common(150)]

# Step 3: Filter only top 150 ingredients
recipes_df_cleaned['filtered_ingredients'] = recipes_df_cleaned['ingredients_cleaned'].apply(
    lambda x: [ing for ing in x if ing in top_ingredients]
)

# Step 4: One-hot encode filtered list
mlb = MultiLabelBinarizer()
ingredient_encoded = pd.DataFrame(
    mlb.fit_transform(recipes_df_cleaned['filtered_ingredients']),
    columns=mlb.classes_
)

# Step 5: Merge back to original dataframe
recipes_df = pd.concat([recipes_df_cleaned.reset_index(drop=True), ingredient_encoded], axis=1)
recipes_df.head()


,name,tags,ingredients,calories,fat_g,protein_g,carbohydrates_g,ingredients_cleaned,filtered_ingredients,all-purpose flour,...,walnuts,water,whipping cream,white pepper,white sugar,white wine,whole wheat flour,worcestershire sauce,yellow onion,zucchini
0,arriba baked winter squash mexican style,"['60-minutes-or-less', 'time-to-make', 'course...","[winter squash, mexican seasoning, mixed spice...",51.5,0.0,2.0,10.88,"[winter squash, mexican seasoning, mixed spice...","[honey, butter, olive oil, salt]",0,...,0,0,0,0,0,0,0,0,0,0
1,a bit different breakfast pizza,"['30-minutes-or-less', 'time-to-make', 'course...","[prepared pizza crust, sausage patty, eggs, mi...",173.4,18.0,22.0,0.00,"[prepared pizza crust, sausage patty, eggs, mi...","[eggs, milk, salt and pepper, cheese]",0,...,0,0,0,0,0,0,0,0,0,0
2,all in the kitchen chili,"['time-to-make', 'course', 'preparation', 'mai...","[ground beef, yellow onions, diced tomatoes, t...",269.8,22.0,39.0,0.00,"[ground beef, yellow onions, diced tomatoes, t...","[ground beef, diced tomatoes, tomato paste, wa...",0,...,0,1,0,0,0,0,0,0,0,0
3,alouette potatoes,"['60-minutes-or-less', 'time-to-make', 'course...","[spreadable cheese with garlic and herbs, new ...",368.1,17.0,14.0,39.78,"[spreadable cheese with garlic and herbs, new ...","[parsley, olive oil, red wine vinegar, salt, p...",0,...,0,0,0,0,0,0,0,0,0,0
4,amish tomato ketchup for canning,"['weeknight', 'time-to-make', 'course', 'main-...","[tomato juice, apple cider vinegar, sugar, sal...",352.9,1.0,3.0,82.98,"[tomato juice, apple cider vinegar, sugar, sal...","[sugar, salt, pepper, dry mustard]",0,...,0,0,0,0,0,0,0,0,0,0


Now, we create the nutrition dataframe

In [8]:
nutrition_df = recipes_df[['name', 'calories', 'fat_g', 'protein_g', 'carbohydrates_g']]
nutrition_df = recipes_df[['name', 'calories', 'fat_g', 'protein_g', 'carbohydrates_g']].rename(columns={
    'name': 'title',
    'fat_g': 'fat',
    'protein_g': 'protein',
    'carbohydrates_g': 'carbohydrates'
})

nutrition_df.head()

,title,calories,fat,protein,carbohydrates
0,arriba baked winter squash mexican style,51.5,0.0,2.0,10.88
1,a bit different breakfast pizza,173.4,18.0,22.0,0.00
2,all in the kitchen chili,269.8,22.0,39.0,0.00
3,alouette potatoes,368.1,17.0,14.0,39.78
4,amish tomato ketchup for canning,352.9,1.0,3.0,82.98


To derive nutrition tags (High Protein, Low Carb, Is Balanced), macronutrient dimensions are used (calories, protein, carbohydrates, fat)
- **High Protein**: 40% or more of calories coming from protein.
- **Low Carb**: 10% or less of calories coming from carbohydrates.
- **Is Balanced**: about 40% calories coming from protein, about 30% calories coming from carbohydrates, about 30% calories coming from fat .

In [9]:
# Dervive high protein dimension
nutrition_df['high protein'] = np.where((nutrition_df['protein']*4) >= (nutrition_df['calories']*0.4), 1, 0)

# Dervive low carb dimension
nutrition_df['low carb'] = np.where((nutrition_df['carbohydrates']*4) <= (nutrition_df['calories']*0.1), 1, 0)

# Dervive is_balanced dimension
nutrition_df['is_balanced'] = np.where(
    ((nutrition_df['protein'] * 4 / nutrition_df['calories'] * 100).between(30, 50)) &
    ((nutrition_df['carbohydrates'] * 4 / nutrition_df['calories'] * 100).between(25, 45)) &
    ((nutrition_df['fat'] * 9 / nutrition_df['calories'] * 100).between(20, 35)),
    1, 0
)

nutrition_df.head()

,title,calories,fat,protein,carbohydrates,high protein,low carb,is_balanced
0,arriba baked winter squash mexican style,51.5,0.0,2.0,10.88,0,0,0
1,a bit different breakfast pizza,173.4,18.0,22.0,0.00,1,1,0
2,all in the kitchen chili,269.8,22.0,39.0,0.00,1,1,0
3,alouette potatoes,368.1,17.0,14.0,39.78,0,0,0
4,amish tomato ketchup for canning,352.9,1.0,3.0,82.98,0,0,0


Select and maintain 150 most popular ingredients only

In [11]:
freq_ingredients = []
with open('../data/files/ingredients.txt', 'r') as f:
    freq_ingredients = [i.strip() for i in f]

selected_columns = ['name'] + freq_ingredients
ingredients_df = recipes_df.loc[:, recipes_df.columns.intersection(selected_columns)]
ingredients_df = ingredients_df.rename(columns={
    'name': 'title'
})
ingredients_df.head()

,title,all-purpose flour,bacon,baking powder,baking soda,balsamic vinegar,banana,bananas,basil,bay leaf,...,walnuts,water,whipping cream,white pepper,white sugar,white wine,whole wheat flour,worcestershire sauce,yellow onion,zucchini
0,arriba baked winter squash mexican style,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,a bit different breakfast pizza,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,all in the kitchen chili,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,alouette potatoes,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,amish tomato ketchup for canning,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now we select the 200 most popular ingredients

In [ ]:
title = ingredients_df['title']
ingredients_df = ingredients_df.select_dtypes(include=['number'])
ingredient_counts = ingredients_df.sum().sort_values(ascending=False)
top_ingredients = ingredient_counts.head(200).index
ingredients_df = ingredients_df[top_ingredients]
ingredients_df['title'] = title
ingredients_df.head()
print(f'🔹 200 most common ingredients: {list(ingredients_df.columns)}')
ingredients_df.head()

🔹 200 most common ingredients: ['salt', 'butter', 'sugar', 'onion', 'water', 'eggs', 'olive oil', 'flour', 'milk', 'garlic cloves', 'pepper', 'brown sugar', 'garlic', 'all-purpose flour', 'baking powder', 'egg', 'salt and pepper', 'parmesan cheese', 'lemon juice', 'baking soda', 'vegetable oil', 'vanilla', 'black pepper', 'cinnamon', 'tomatoes', 'sour cream', 'garlic powder', 'vanilla extract', 'oil', 'honey', 'onions', 'cream cheese', 'celery', 'cheddar cheese', 'unsalted butter', 'soy sauce', 'mayonnaise', 'paprika', 'chicken broth', 'worcestershire sauce', 'extra virgin olive oil', 'cornstarch', 'carrots', 'parsley', 'chili powder', 'bacon', 'carrot', 'potatoes', 'nutmeg', 'cayenne pepper', 'granulated sugar', 'ground cumin', 'green onions', 'red onion', 'walnuts', 'pecans', 'dijon mustard', 'green onion', 'kosher salt', 'powdered sugar', 'fresh lemon juice', 'heavy cream', 'margarine', 'orange juice', 'zucchini', 'raisins', 'red bell pepper', 'tomato sauce', 'fresh cilantro', 'chic

,salt,butter,sugar,onion,water,eggs,olive oil,flour,milk,garlic cloves,...,scallions,nuts,monterey jack cheese,peanut butter,banana,yellow onion,half-and-half,seasoning salt,red pepper,title
0,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,arriba baked winter squash mexican style
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,a bit different breakfast pizza
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,all in the kitchen chili
3,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,alouette potatoes
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,amish tomato ketchup for canning


#### Combining datasets
The dataframes with nutrition and ingredient details are now merged into one

In [12]:
recipes_df = pd.merge(nutrition_df, ingredients_df, how='inner', on='title')
recipes_df.head()
recipes_df.head()

,title,calories,fat,protein,carbohydrates,high protein,low carb,is_balanced,all-purpose flour,bacon,...,walnuts,water,whipping cream,white pepper,white sugar,white wine,whole wheat flour,worcestershire sauce,yellow onion,zucchini
0,arriba baked winter squash mexican style,51.5,0.0,2.0,10.88,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,a bit different breakfast pizza,173.4,18.0,22.0,0.00,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,all in the kitchen chili,269.8,22.0,39.0,0.00,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,alouette potatoes,368.1,17.0,14.0,39.78,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,amish tomato ketchup for canning,352.9,1.0,3.0,82.98,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5) Remove duplicates

In [13]:
if recipes_df.duplicated().sum() != 0:
    recipes_df.drop_duplicates(inplace=True)
recipes_df.shape
print(recipes_df.columns.values.tolist())

['title', 'calories', 'fat', 'protein', 'carbohydrates', 'high protein', 'low carb', 'is_balanced', 'all-purpose flour', 'bacon', 'baking powder', 'baking soda', 'balsamic vinegar', 'banana', 'bananas', 'basil', 'bay leaf', 'bay leaves', 'black beans', 'black pepper', 'boiling water', 'breadcrumbs', 'brown sugar', 'butter', 'buttermilk', 'canola oil', 'carrot', 'carrots', 'cayenne pepper', 'celery', 'cheddar cheese', 'cheese', 'chicken breasts', 'chicken broth', 'chicken stock', 'chili powder', 'cider vinegar', 'cilantro', 'cinnamon', "confectioners' sugar", 'cornstarch', 'cream cheese', 'crushed red pepper flakes', 'cumin', 'curry powder', 'diced tomatoes', 'dijon mustard', 'dry mustard', 'dry white wine', 'egg', 'egg whites', 'egg yolks', 'eggs', 'extra virgin olive oil', 'feta cheese', 'flour', 'fresh cilantro', 'fresh lemon juice', 'garlic', 'garlic cloves', 'garlic powder', 'garlic salt', 'ginger', 'granulated sugar', 'green bell pepper', 'green onion', 'green onions', 'green pepp

## 6) Save dataset as CSV
The dataset 2 is not saved as CSV to import in and marge with original dataframe

In [ ]:
# Export to CSV
recipes_df.to_csv('../data/processed/recipes_2.csv', index=False)

: 